In [ ]:
w!pip install pyarrow

In [ ]:
!pip install certifi

In [ ]:
!pip install earthengine-api geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.4 MB/s eta 0:00:00


In [ ]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import os

SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/TeamProject2/"
os.makedirs(SAVE_PATH, exist_ok=True)

# Inisialisasi Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='gen-lang-client-0166687433')

# Hitung Luas Lahan per Provinsi
def compute_luas_lahan():
    landcover = ee.ImageCollection('ESA/WorldCover/v100').first().select('Map')
    provinsi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', 'Indonesia'))

    classNames = ['Hutan', 'Pertanian', 'Padang Rumput', 'Semak Belukar', 'Perkotaan', 'Lahan Basah', 'Perairan', 'Lahan Kritis']
    classValues = [10, 20, 30, 40, 50, 60, 70, 80]

    def compute_luas(feature):
        stats = landcover.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=feature.geometry(),
            scale=10,
            maxPixels=1e13
        )
        histogram = ee.Dictionary(stats.get('Map'))
        luas_dict = ee.Dictionary({
            f'Luas_{classNames[i]}_ha': ee.Number(histogram.get(str(classValues[i]), 0)).multiply(100).divide(10000)
            for i in range(len(classValues))
        })
        return feature.setMulti(luas_dict.combine({'Provinsi': feature.get('ADM1_NAME')}))

    provinsi_luas = provinsi.map(compute_luas)
    features = provinsi_luas.getInfo()['features']
    data_list = [feature['properties'] for feature in features]
    df = pd.DataFrame(data_list)
    return df

# Ambil Data dari GeoJSON
def source_data_from_geojson(file_path):
    try:
        df_geojson = gpd.read_file(file_path)
        df_geojson = df_geojson.dropna().drop_duplicates()
    except Exception as e:
        df_geojson = pd.DataFrame()
    return df_geojson

# Ambil Data dari Website
def source_data_from_webpage(web_page_url, matching_keyword):
    try:
        df_html = pd.read_html(web_page_url, match=matching_keyword)
        df_html = df_html[0]
    except Exception as e:
        df_html = pd.DataFrame()
    return df_html

# Simpan ke File CSV
def save_to_csv(df, filename):
    output_file = os.path.join(SAVE_PATH, filename)
    try:
        df.to_csv(output_file, index=False, sep=",", encoding="utf-8")
        print(f"Data berhasil disimpan di: {output_file}")
    except Exception as e:
        print(f"Error saat menyimpan {filename}: {e}")

def extract_all_data():
    df_geo = compute_luas_lahan()
    save_to_csv(df_geo, "data_geojson.csv")

    url_emisi = "https://statistik.menlhk.go.id/sisklhkX/data_statistik/ppi/table7_8"
    url_luas = "https://statistik.menlhk.go.id/sisklhkX/data_statistik/ppi/table7_6"
    df_emisi_web = source_data_from_webpage(url_emisi, "Provinsi")
    df_luas_web = source_data_from_webpage(url_luas, "Provinsi")
    save_to_csv(df_emisi_web, "emisi_karbon_web.csv")
    save_to_csv(df_luas_web, "luas_kebakaran_web.csv")

    return df_geo, df_emisi_web, df_luas_web

extract_all_data()


Data berhasil disimpan di: /content/drive/MyDrive/Colab Notebooks/TeamProject2/data_geojson.csv
Data berhasil disimpan di: /content/drive/MyDrive/Colab Notebooks/TeamProject2/emisi_karbon_web.csv
Data berhasil disimpan di: /content/drive/MyDrive/Colab Notebooks/TeamProject2/luas_kebakaran_web.csv


(    ADM0_CODE  ADM0_NAME  ADM1_CODE                   ADM1_NAME DISP_AREA  \
 0         116  Indonesia      73615                      Maluku        NO   
 1         116  Indonesia      73616                Maluku Utara        NO   
 2         116  Indonesia      73609             Bangka Belitung        NO   
 3         116  Indonesia      73610                      Banten        NO   
 4         116  Indonesia      73611                   Gorontalo        NO   
 5         116  Indonesia      73612                 Papua Barat        NO   
 6         116  Indonesia      73613                  Jawa Barat        NO   
 7         116  Indonesia      73617                       Papua        NO   
 8         116  Indonesia      73621              Sulawesi Utara        NO   
 9         116  Indonesia      73622            Sumatera Selatan        NO   
 10        116  Indonesia      73614              Kepulauan-riau        NO   
 11        116  Indonesia      73618                        Riau